**Artificial Neural Network - Churn Modelling for Bank Customers**

In [67]:
import numpy as np
import pandas as pd
import tensorflow as tf
import joblib

In [68]:
tf.__version__

'2.10.0'

**Data Preprocessing**

In [69]:
dataset = pd.read_csv('../resources/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [70]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [71]:
print(y)

[1 0 1 ... 1 1 0]


**Encode Categorical Data**

In [72]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2]) #label encode gender column
# Saving the LabelEncoder
joblib.dump(le, '../encoders/label_encoder.joblib')

['../encoders/label_encoder.joblib']

In [73]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encode Geo Col

In [74]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

#Save encoder so we can load it later in the script
encoder = ct.named_transformers_['encoder']
joblib.dump(encoder, '../encoders/onehot_encoder.joblib')

['../encoders/onehot_encoder.joblib']

In [75]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [76]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

**Feature Scaling**

In [77]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
joblib.dump(sc, '../encoders/scaler.bin')

['../encoders/scaler.bin']

**Building the ANN**

In [80]:
ann = tf.keras.models.Sequential()

In [81]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) # add first hidden layer with 6 neurons and the rectifier activation function

In [82]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [83]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

**Compiling the ANN**

In [84]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [85]:
ann.fit(X_train, y_train, batch_size = 32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5397 - accuracy: 0.7664
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4540 - accuracy: 0.8005
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4382 - accuracy: 0.8074
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4328 - accuracy: 0.8086
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4292 - accuracy: 0.8104
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4259 - accuracy: 0.8126
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4235 - accuracy: 0.8145
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4203 - accuracy: 0.8163
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4174 - accuracy: 0.8204
Epoch 10/100
250/250 [==============================] - 0s 996us/step - loss: 0.4138 - accu

**Make a single prediction**

In [86]:
print(ann.predict(sc.transform([[1.0, 0.0, 0.0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 69ms/step
[[False]]


**Predict Test Set**

In [87]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)),1))

63/63 [==============================] - 0s 842us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


**View Results**

In [88]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1503   92]
 [ 188  217]]


0.86

**Save Model as local file**

In [89]:
ann.save('../model/ann-churn-model.h5')

In [90]:
loaded_model = tf.keras.models.load_model('../model/ann-churn-model.h5')

**Generate a prediction from a JSON payload**

In [91]:
import json
file=open("../resources/json_payload.txt","r")
json_payload = json.loads(file.read())
print(json_payload)
##loop over values and append to 2d np array
X_new = []
for value in json_payload.values():
    X_new.append(value)

X_new = np.array([X_new])
print(X_new)

{'CreditScore': 597, 'Geography': 'Spain', 'Gender': 'Female', 'Age': 34, 'Tenure': 2, 'Balance': 0, 'NumOfProducts': 1, 'HasCrCard': 1, 'IsActiveMember': 1, 'EstimatedSalary': 71348.88}
[['597' 'Spain' 'Female' '34' '2' '0' '1' '1' '1' '71348.88']]


Load the label encoder to encode the country

In [92]:
saved_labelencoder = joblib.load('../encoders/label_encoder.joblib')
X_new[:, 2] = saved_labelencoder.transform(X_new[:, 2]) #label encode gender column
print(X_new)

[['597' 'Spain' '0' '34' '2' '0' '1' '1' '1' '71348.88']]


In [93]:
from sklearn.preprocessing import OneHotEncoder
saved_encoder = joblib.load('../encoders/onehot_encoder.joblib')
encoded_values = saved_encoder.transform(X_new[:, [1]]).toarray()
X_new = np.delete(X_new, 1, axis=1) #remove nonencoded country column
X_new = np.hstack((encoded_values, X_new)) #concatenate because OneHotEncoder pushes encoded values to beginning of array
print(X_new)

[['0.0' '0.0' '1.0' '597' '0' '34' '2' '0' '1' '1' '1' '71348.88']]


**Before Save Prediction**

In [94]:
print(ann.predict(sc.transform(X_new)))

1/1 [==============================] - 0s 32ms/step
[[0.21961598]]


**After Save Prediction**

In [95]:
new_sc = joblib.load('../encoders/scaler.bin')
print(loaded_model.predict(sc.transform(X_new)))

1/1 [==============================] - 0s 62ms/step
[[0.21961598]]
